In [8]:
pip install unsloth transformers accelerate



SyntaxError: invalid syntax (<ipython-input-8-029c7f388903>, line 1)

In [7]:
!pip install fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel

<ipython-input-2-bb3d42e59f17>:3: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
import gc
gc.collect()

torch.cuda.empty_cache()

In [5]:
#del model, tokenizer

In [4]:
model_name = "unsloth/llama-3-8b-instruct"
model, tokenizer = FastLanguageModel.from_pretrained(model_name, load_in_4bit=True)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [43]:
def chat_with_cuy(user_input):
    prompt = f"""Eres un cuy experto en finanzas que ayuda a los usuarios con información clara y precisa sobre los productos y servicios del BCP en Perú.

    - Responde de forma directa, sin repetir la pregunta del usuario.
    - No generes contexto adicional, solo responde con información relevante.
    - Siempre eres amigable y dices "¡Cui!" al inicio de cada respuesta.
    - Si la pregunta no es sobre finanzas, di: "¡Cui! Solo sé sobre finanzas".
    - Puedes responder con jergas peruanas.

    Recuerda esto:
    - Solo el 1% de mujeres peruanas accede a créditos hipotecarios (Equifax, 2024).
    - El 42.2% de las mujeres peruanas son trabajadoras independientes con ingresos variables (INEI, 2023).
    - El 61% de las usuarias de banca móvil ha recibido intentos de fraude (BCRP, 2023).
    - Solo el 34% de las mujeres usa banca móvil debido a falta de información (BCRP).

    Usuario: {user_input}
    Cui responde:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    output = model.generate(**inputs, max_new_tokens=50, do_sample=False, temperature=0.1, repetition_penalty=1.1)

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    response = response.split("Cui responde:")[-1].strip()

    first_line = response.split("\n")[0].strip()

    return f"¡Cui! {first_line}"


In [ ]:
# probando el chatbot
while True:
    user_input = input("Tú: ")
    if user_input.lower() in ["salir", "exit"]:
        break
    response = chat_with_cuy(user_input)
    print("TuKuy:", response)

In [13]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import uvicorn

In [10]:
!ngrok config add-authtoken 2ujyYcXelunomFV31D7Itf2wNac_28JuwPVd3VTpzQJDHvAqV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [44]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class UserInput(BaseModel):
    text: str

@app.post("/chat/")
def chat(user_input: UserInput):
    response = chat_with_cuy(user_input.text)
    return {"response": response}

public_url = ngrok.connect(8000)
print("La API está en:", public_url)

La API está en: NgrokTunnel: "https://7f9d-34-16-218-94.ngrok-free.app" -> "http://localhost:8000"


In [21]:
!pkill -f ngrok


In [45]:
import uvicorn
import nest_asyncio

nest_asyncio.apply()

uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [197]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     34.58.220.57:0 - "POST /chat/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [197]
